In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            print("Calc_reward target bonus: ", 5000 + 5000 * (1 - abs(self.env.state.speed)))
            return 5000 + 5000 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -2
            
            if len(self.env.distances) > 0 and len(self.env.history) > 1:
                target_diff = self.env.distances[-1] - self.env.get_target_distance(False)
                
                gps_diff_0 = self.env.history[-1].gps_actual[0] - self.env.state.gps_actual[0]
                gps_diff_1 = self.env.history[-1].gps_actual[1] - self.env.state.gps_actual[1]
                
                #print("history: ", self.env.history[-1].gps_actual[0])
                #print("actual:  " , self.env.state.gps_actual[0])
                
                actual_diff = math.sqrt(gps_diff_0**2 + gps_diff_1**2)
                
                if actual_diff > 0:
                    diff_rew = 4 * ( target_diff  / actual_diff)
                else:
                    diff_rew = 0
                
                
                
                if abs(diff_rew) > 4:
                    print("============= Quotient > 1 !? ==========")
                    print("Target_diff: ", target_diff)
                    print("Actual_diff: ", actual_diff)
                    print("diff_rew:    ", diff_rew)
                    
                if diff_rew < 0:
                    diff_rew = diff_rew * 2
                
                #print("Target_diff: ", target_diff)
                #print("Actual_diff: ", actual_diff)
                #print("diff_rew:    ", diff_rew)  
                    
  
                reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -5
        
            if self.env.state.touching:
                reward += -100
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary, reward: ", self.env.total_reward)
            return True
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            return True
        if self.env.total_reward > 25000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 10
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
                action_class = wg.WbtActContinuous(config=config, bound=0.3, relative = True),
                config=config)

../../backend/webotsgym/env/webotenv.py:111: UserWarning: Relative property of action class is overwritten by config.relative_action.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
model_name = "3x3_reach_target_v7_fabi"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 5000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
while True:
    model.learn(total_timesteps=400000)
    model.save("./model_{}".format(model_name))


Calc_reward target bonus:  9953.726520761847
target reached, reward:  -2676.955300794289
Calc_reward target bonus:  9937.571734189987
target reached, reward:  -2082.2787528221725
Calc_reward target bonus:  9936.06608826667
target reached, reward:  -4301.197645979009
reward boundary, reward:  -5011.78496217159
Calc_reward target bonus:  9967.884996440262
target reached, reward:  -2786.3158705195274
Calc_reward target bonus:  9955.11980727315
target reached, reward:  -502.0220114805631
reward boundary, reward:  -5060.75909023258
reward boundary, reward:  -5081.787975784749
Calc_reward target bonus:  9966.926760971546
target reached, reward:  -2586.0843372386985
reward boundary, reward:  -5085.385354616509
Calc_reward target bonus:  9957.23347645253
target reached, reward:  -46.09213707020795
reward boundary, reward:  -5092.360660905982
Calc_reward target bonus:  9958.506072871387
target reached, reward:  -21.650217047281217
reward boundary, reward:  -5063.640393148449
reward boundary, r

reward boundary, reward:  -5042.063128962801
reward boundary, reward:  -5028.057314338788
reward boundary, reward:  -5033.1948825244
reward boundary, reward:  -5068.149195999297
reward boundary, reward:  -5081.6985632525675
Calc_reward target bonus:  9967.309110797942
target reached, reward:  -797.6626652729066
reward boundary, reward:  -5011.515769642569
reward boundary, reward:  -5005.151930431336
reward boundary, reward:  -5066.437442860197
Calc_reward target bonus:  9966.89832303673
target reached, reward:  -23.982540690070778
reward boundary, reward:  -5038.656100303745
reward boundary, reward:  -5003.951506507328
reward boundary, reward:  -5026.908420455407
reward boundary, reward:  -5035.388537122622
reward boundary, reward:  -5008.318367698914
reward boundary, reward:  -5084.663768599564
reward boundary, reward:  -5013.596501942358
reward boundary, reward:  -5110.403896296842
reward boundary, reward:  -5037.72859683909
reward boundary, reward:  -5011.176263696755
reward boundar

reward boundary, reward:  -5049.521082850137
reward boundary, reward:  -5003.095661683788
reward boundary, reward:  -5034.249043041475
Calc_reward target bonus:  9937.494383193552
target reached, reward:  -2360.6611302976607
reward boundary, reward:  -5070.77984760654
reward boundary, reward:  -5026.209842677041
reward boundary, reward:  -5010.75440818401
reward boundary, reward:  -5078.995091921017
Calc_reward target bonus:  9930.003616027534
target reached, reward:  -351.51725900131635
reward boundary, reward:  -5003.474638167398
Calc_reward target bonus:  9934.423225931823
target reached, reward:  5.658348175192548
reward boundary, reward:  -5021.128803858003
reward boundary, reward:  -5112.630067149406
reward boundary, reward:  -5077.642749479575
reward boundary, reward:  -5023.832424526758
reward boundary, reward:  -5108.466488634694
reward boundary, reward:  -5043.6875494953365
Calc_reward target bonus:  9946.596841327846
target reached, reward:  -1979.3987200624722
reward bounda

reward boundary, reward:  -5067.03695492444
reward boundary, reward:  -5025.0277149855
reward boundary, reward:  -5066.9132642747
reward boundary, reward:  -5055.466332613996
reward boundary, reward:  -5102.410536925746
reward boundary, reward:  -5063.545871670744
reward boundary, reward:  -5011.339839194232
reward boundary, reward:  -5093.001242713122
reward boundary, reward:  -5048.574140879544
reward boundary, reward:  -5006.612299883988
reward boundary, reward:  -5091.173440295323
reward boundary, reward:  -5097.13941066977
reward boundary, reward:  -5010.972974341992
reward boundary, reward:  -5002.563638546232
Calc_reward target bonus:  9964.677125681192
target reached, reward:  -1326.5174111517438
Calc_reward target bonus:  9941.690191626549
target reached, reward:  -2992.1969892706197
reward boundary, reward:  -5015.44263143412
reward boundary, reward:  -5000.81831439608
reward boundary, reward:  -5057.157663581482
reward boundary, reward:  -5038.608710151835
reward boundary, r

reward boundary, reward:  -5044.675102101811
reward boundary, reward:  -5003.408942249641
reward boundary, reward:  -5013.184551993358
reward boundary, reward:  -5015.40983051562
reward boundary, reward:  -5017.169744424888
reward boundary, reward:  -5015.6803240417275
reward boundary, reward:  -5052.470718146366
reward boundary, reward:  -5002.365487935991
reward boundary, reward:  -5033.077094776051
reward boundary, reward:  -5065.638350508969
reward boundary, reward:  -5012.604811101952
reward boundary, reward:  -5013.309270584576
Calc_reward target bonus:  9947.955547831953
target reached, reward:  -771.6067505054159
reward boundary, reward:  -5085.527431797205
reward boundary, reward:  -5081.870857952119
Calc_reward target bonus:  9950.122563168406
target reached, reward:  -474.3383577849624
reward boundary, reward:  -5084.023378141542
reward boundary, reward:  -5091.608717116634
reward boundary, reward:  -5030.489141400084
reward boundary, reward:  -5046.577027433651
reward bound

...

Calc_reward target bonus:  9952.781978063285
target reached, reward:  -88.10565466712492
Calc_reward target bonus:  9923.580484464765
target reached, reward:  -199.36980059039448
Calc_reward target bonus:  9963.057390414178
target reached, reward:  -2658.3865581479918
Calc_reward target bonus:  9926.449940539896
target reached, reward:  -152.57662778546208
Calc_reward target bonus:  9932.12919216603
target reached, reward:  -3497.075976560073
Calc_reward target bonus:  9976.168975699693
target reached, reward:  -107.96940084335603
Calc_reward target bonus:  9933.150778524578
target reached, reward:  -252.60309027928844
reward boundary, reward:  -5004.1280259677915
Calc_reward target bonus:  9973.79369335249
target reached, reward:  -41.220934605682444
Calc_reward target bonus:  9933.613394387066
target reached, reward:  -246.31223007420093
Calc_reward target bonus:  9973.170389421284
target reached, reward:  -162.44413562654296
Calc_reward target bonus:  9951.749024912715
target reache

reward boundary, reward:  -5040.9935820967
Calc_reward target bonus:  9955.304088070989
target reached, reward:  -2149.1129437182167
reward boundary, reward:  -5083.413891011284
Calc_reward target bonus:  9947.760724462569
target reached, reward:  -1084.7035496839358
reward boundary, reward:  -5007.220064093132
Calc_reward target bonus:  9953.839764930308
target reached, reward:  -432.61878580539707
Calc_reward target bonus:  9971.619539428502
target reached, reward:  -418.1186462984857
reward boundary, reward:  -5074.804288515831
Calc_reward target bonus:  9943.1770760566
target reached, reward:  -2255.3049947896125
reward boundary, reward:  -5057.174010350838
Calc_reward target bonus:  9949.511238373816
target reached, reward:  -499.23282512703605
Calc_reward target bonus:  9951.498890295625
target reached, reward:  -657.6793401653186
Calc_reward target bonus:  9970.638900995255
target reached, reward:  -635.9491644408961
Calc_reward target bonus:  9932.321887463331
target reached, r

reward boundary, reward:  -5105.75935228465
Calc_reward target bonus:  9948.45234323293
target reached, reward:  -1346.670162813232
Calc_reward target bonus:  9930.557166226208
target reached, reward:  -1487.462992585109
Calc_reward target bonus:  9950.719592161477
target reached, reward:  -125.99792508931871
reward boundary, reward:  -5055.630071723073
Calc_reward target bonus:  9948.739572428167
target reached, reward:  -1099.2635840123976
reward boundary, reward:  -5086.579258008681
reward boundary, reward:  -5058.605486357596
Calc_reward target bonus:  9939.594734460115
target reached, reward:  -1959.4618312499358
reward boundary, reward:  -5078.042829472979
Calc_reward target bonus:  9992.100147064775
target reached, reward:  -246.17006180574074
reward boundary, reward:  -5093.064215799598
Calc_reward target bonus:  9946.413305588067
target reached, reward:  -1847.2726607230709
Calc_reward target bonus:  9990.285818930715
target reached, reward:  -1588.737722752491
Calc_reward tar

Calc_reward target bonus:  9940.541028045118
target reached, reward:  -65.58991805248569
Calc_reward target bonus:  9966.069832444191
target reached, reward:  -197.41066640360214
reward boundary, reward:  -5001.21856127713
reward boundary, reward:  -5005.280560596286
reward boundary, reward:  -5107.1276578541765
reward boundary, reward:  -5001.088031382465
Calc_reward target bonus:  9975.398825481534
target reached, reward:  -587.2326907808688
reward boundary, reward:  -5097.427911816226
reward boundary, reward:  -5004.812691640019
reward boundary, reward:  -5094.930573566293
reward boundary, reward:  -5030.738163791931
Calc_reward target bonus:  9965.10801371187
target reached, reward:  -1679.2412252004738
Calc_reward target bonus:  9952.207333408296
target reached, reward:  -1194.3200695388898
Calc_reward target bonus:  9942.27396324277
target reached, reward:  -1025.501706107553
reward boundary, reward:  -5074.649094956677
reward boundary, reward:  -5038.833407789647
Calc_reward tar

KeyboardInterrupt: 